# g5_v032
エリア×機種のcost_min重み

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 
import scipy.interpolate
import scipy.sparse

In [2]:
area_labeling = pd.read_csv('../output/prep/area_labeling/result.csv')

g1 = list(area_labeling[area_labeling['g']==1]['collectionName'])
g2 = list(area_labeling[area_labeling['g']==2]['collectionName'])
g3 = list(area_labeling[area_labeling['g']==3]['collectionName'])
g4 = list(area_labeling[area_labeling['g']==4]['collectionName'])
g5 = list(area_labeling[area_labeling['g']==5]['collectionName'])

In [3]:
target = g5

In [4]:
ro_th = 50 # 相対移動距離をもとにした異常値除去の閾値
rog_th = 2 # ground_truthをもとにした異常値除去の閾値
rp_rate = 0.5 # 相対座標を混ぜる割合

# ground_truthをもとにした異常値除去を行うcollection
rog_target = g3 + g4 + g5

In [5]:
rb_dir = '../output/exp104/'
sp0_dir = '../output/prep/speed0_pred_v001'
degree_dir = '../output/prep/degree_pred_v004'
dist_dir = '../output/prep/distance_pred_v002'

In [6]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [7]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [8]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [9]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [10]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)
    
def vis(df, phone):
    gt = get_ground_truth()
    gt['phone'] = gt['collectionName'] + '_' + gt['phoneName']
    tmp1 = df[df['phone']==phone].copy()
    tmp2 = gt[gt['phone']==phone].copy()
    
    tmp1['phoneName'] = 'train'
    tmp2['phoneName'] = 'gt'
    
    tmp = pd.concat([tmp1, tmp2])
    tmp['collectionName'] = 'all'
    
    lat_center = tmp['latDeg'].mean()
    lng_center = tmp['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
        
    visualize_trafic(tmp, center)

In [11]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [12]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [13]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [14]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = get_ground_truth()
    return base_train, base_test, sample_sub, ground_truth

# 相対移動距離をもとにした外れ値除去

In [15]:
train, test, sub, gt = get_data()
train = train[train['collectionName'].isin(target)]

In [16]:
rb = pd.read_csv(f'{rb_dir}/result.csv')

rb = rb.rename(columns={'latDeg':'latDeg_rb', 'lngDeg':'lngDeg_rb'})

train = train.merge(rb[['millisSinceGpsEpoch', 'phone', 'latDeg_rb', 'lngDeg_rb']], on=['millisSinceGpsEpoch', 'phone'], how='left')

idx = train[~train['latDeg_rb'].isnull()].index
train.loc[idx, 'latDeg'] = train.loc[idx, 'latDeg_rb']
train.loc[idx, 'lngDeg'] = train.loc[idx, 'lngDeg_rb']

In [17]:
def add_distance_diff(df):
    df['latDeg_prev'] = df['latDeg'].shift(1)
    df['latDeg_next'] = df['latDeg'].shift(-1)
    df['lngDeg_prev'] = df['lngDeg'].shift(1)
    df['lngDeg_next'] = df['lngDeg'].shift(-1)
    df['phone_prev'] = df['phone'].shift(1)
    df['phone_next'] = df['phone'].shift(-1)
    
    df['dist_prev'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_prev'], df['lngDeg_prev'])
    df['dist_next'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_next'], df['lngDeg_next'])
    
    df.loc[df['phone']!=df['phone_prev'], ['latDeg_prev', 'lngDeg_prev', 'dist_prev']] = np.nan
    df.loc[df['phone']!=df['phone_next'], ['latDeg_next', 'lngDeg_next', 'dist_next']] = np.nan
    
    return df

In [18]:
# reject outlier
train = add_distance_diff(train)
train.loc[((train['dist_prev'] > ro_th) & (train['dist_next'] > ro_th)), ['latDeg', 'lngDeg']] = np.nan
train.to_csv(OUTPUT + '/train_ro.csv', index=False)

# ground_truthを基準にした外れ値除去

In [19]:
def get_osmnx_data():
    p = pathlib.Path(INPUT)
    files = list(p.glob('prep/osmnx/*.csv'))

    dfs = []
    for file in files:
        dfs.append(pd.read_csv(file))
    osmnx = pd.concat(dfs)

    return osmnx

In [20]:
def remove_based_on_gt(target_df, target_gt, th):
    osmnx_data = get_osmnx_data()
    target_gt = target_gt.append(osmnx_data)
    
    for idx in target_df.index:
        lat = target_df.at[idx, 'latDeg']
        lng = target_df.at[idx, 'lngDeg']
        collection = target_df.at[idx, 'collectionName']
        
        if np.isnan(lat):
            continue
        else:
            target_gt['latDeg_pred'] = lat
            target_gt['lngDeg_pred'] = lng
            target_gt['dist'] = calc_haversine(target_gt['latDeg'], target_gt['lngDeg'], target_gt['latDeg_pred'], target_gt['lngDeg_pred'])
            closest_dist = target_gt['dist'].min()
            if closest_dist > th:
                target_df.at[idx, 'latDeg'] = np.nan
                target_df.at[idx, 'lngDeg'] = np.nan
    return target_df

In [21]:
def interpolate_remove_point(df):
    out_df = pd.DataFrame()
    collections = df['collectionName'].unique()
    
    for collection in collections:
        tmp = df[df['collectionName']==collection].copy()
        tmp = tmp.sort_values('millisSinceGpsEpoch')
        tmp = tmp.reset_index().set_index('millisSinceGpsEpoch')
        tmp[['latDeg', 'lngDeg']] = tmp[['latDeg', 'lngDeg']].interpolate(method='index', limit_area='inside')
        tmp = tmp.sort_values('index')
        tmp = tmp.reset_index().set_index('index')
        out_df = out_df.append(tmp)
    return out_df

In [22]:
def interpolate_by_rel(df):
    df_idx = list(df.index)
    df['lat_interp_f'] = df['latDeg']
    df['lng_interp_f'] = df['lngDeg']
    df['lat_interp_b'] = df['latDeg']
    df['lng_interp_b'] = df['lngDeg']
    
    for idx in df_idx:
        if idx == df_idx[0]:
            continue
        if df.at[idx, 'phone'] != df.at[idx-1, 'phone']:
            continue
        if ~np.isnan(df.at[idx, 'latDeg']):
            continue
            
        lat_prev = df.at[idx-1, 'lat_interp_f']
        lng_prev = df.at[idx-1, 'lng_interp_f']
        lat_diff = df.at[idx-1, 'lat_diff']
        lng_diff = df.at[idx-1, 'lng_diff']
        
        if ~np.isnan(lat_prev):
            df.at[idx, 'lat_interp_f'] = lat_prev + lat_diff
            df.at[idx, 'lng_interp_f'] = lng_prev + lng_diff
    
    df_idx.reverse()
    for idx in df_idx:
        if idx == df_idx[0]:
            continue
        if df.at[idx, 'phone'] != df.at[idx+1, 'phone']:
            continue
        if ~np.isnan(df.at[idx, 'latDeg']):
            continue
            
        lat_prev = df.at[idx+1, 'lat_interp_b']
        lng_prev = df.at[idx+1, 'lng_interp_b']
        lat_diff = df.at[idx+1, 'lat_diff']
        lng_diff = df.at[idx+1, 'lng_diff']
        
        if ~np.isnan(lat_prev):
            df.at[idx, 'lat_interp_b'] = lat_prev - lat_diff
            df.at[idx, 'lng_interp_b'] = lng_prev - lng_diff
            
    df['latDeg'] = (df['lat_interp_f'] + df['lat_interp_b']) / 2
    df['lngDeg'] = (df['lng_interp_f'] + df['lng_interp_b']) / 2
            
    return df

In [23]:
rog_df = train[train['collectionName'].isin(rog_target)].copy()
rog_gt = gt[gt['collectionName'].isin(rog_target)].copy()
rog_df = remove_based_on_gt(rog_df, rog_gt, rog_th)
train.loc[rog_df.index, ['latDeg', 'lngDeg']] = rog_df[['latDeg', 'lngDeg']]

# speed0の処理

In [24]:
def sp0_process(df, sp0_result):
    df = df.merge(sp0_result, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    df['group'] = df.groupby('phone').cumcount()
    df.loc[(df['group']>0)&(df['isSpeed0']==1),'group'] = np.nan
    df['group'] = df['group'].fillna(method='ffill')
    df[['latDeg', 'lngDeg']] = df.groupby(['phone', 'group'])[['latDeg','lngDeg']].transform('mean')
    return df

In [25]:
train_sp0_pred = pd.read_csv(f'{sp0_dir}/train_sp0_pred.csv')
train = sp0_process(train, train_sp0_pred)

# 補間

In [26]:
dp_train = pd.read_csv('../output/prep/rel_pred_v001/train_result.csv')
train = train.merge(dp_train[['millisSinceGpsEpoch', 'phone', 'lat_diff', 'lng_diff']], on=['millisSinceGpsEpoch', 'phone'], how='left')
train.loc[train['isSpeed0']==1, ['lat_diff', 'lng_diff']] = 0

In [27]:
train = interpolate_by_rel(train)

# kalmanフィルタ

In [28]:
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in unique_paths:
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

In [29]:
train = apply_kf_smoothing(train)

In [30]:
res = train_result(train)
print(res.score)
display(res.err)

7.616210438858158


,percentile50,percentile95,p50_p90_mean
phone,,,
2021-04-22-US-SJC-1_Pixel4,3.564936,15.058229,9.311583
2021-04-22-US-SJC-1_SamsungS20Ultra,4.011106,12.307432,8.159269
2021-04-28-US-SJC-1_Pixel4,2.153498,10.638001,6.395750
2021-04-28-US-SJC-1_SamsungS20Ultra,2.987857,9.842893,6.415375
2021-04-29-US-SJC-2_Pixel4,1.985037,12.401111,7.193074
2021-04-29-US-SJC-2_SamsungS20Ultra,2.868065,13.576361,8.222213


In [31]:
def cost_minimization(df, b):
    xy_hat = df[['latDeg', 'lngDeg']].to_numpy()
    delta_xy_hat = df[['lat_diff', 'lng_diff']].fillna(method='ffill')[:-1].to_numpy()

    N = xy_hat.shape[0]
    alpha = 0.01 * np.ones(N)
    beta  = b * np.ones(N-1)

    A = scipy.sparse.spdiags(alpha, [0], N, N)
    B = scipy.sparse.spdiags(beta, [0], N-1, N-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

    Q = A + (D.T @ B @ D)
    c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
    xy_star = scipy.sparse.linalg.spsolve(Q, c)

    return xy_star

In [32]:
def cost_min_objective(trial):
    cost_min_beta_w = trial.suggest_uniform('cost_min_beta_w', 0.01, 0.5)
    pixel4_coef = trial.suggest_uniform('pixel4_coef', 0.1, 1)
    samsungs20ultra_coef = trial.suggest_uniform('samsungs20ultra_coef', 0.1, 1)
    train_ = train.copy()
    
    phones = train['phone'].unique()
    for phone in phones:
        
        phonename = phone.split('_')[1]
        if phonename == 'Pixel4':
            w = cost_min_beta_w * pixel4_coef
        elif phonename == 'SamsungS20Ultra':
            w = cost_min_beta_w * samsungs20ultra_coef
            
        idx = train_[train_['phone']==phone].index
        train_.loc[idx, ['latDeg', 'lngDeg']] = cost_minimization(train_.loc[idx], w)
    score = train_result(train_).score
    return score

In [33]:
study = optuna.create_study()
study.optimize(cost_min_objective, n_trials=50)
cost_min_beta_w = study.best_params['cost_min_beta_w']
pixel4_coef = study.best_params['pixel4_coef']
samsungs20ultra_coef = study.best_params['samsungs20ultra_coef']
print(cost_min_beta_w)

[I 2021-08-02 10:39:09,530] A new study created in memory with name: no-name-f714218a-d4aa-4b8a-af5f-6080ae1526c9
[I 2021-08-02 10:39:09,901] Trial 0 finished with value: 7.206494463350026 and parameters: {'cost_min_beta_w': 0.2499109366518664, 'pixel4_coef': 0.21386175256774165, 'samsungs20ultra_coef': 0.7787345267709814}. Best is trial 0 with value: 7.206494463350026.
[I 2021-08-02 10:39:10,259] Trial 1 finished with value: 7.1314102160395185 and parameters: {'cost_min_beta_w': 0.35674107000242516, 'pixel4_coef': 0.8705898611729849, 'samsungs20ultra_coef': 0.2994807369830229}. Best is trial 1 with value: 7.1314102160395185.
[I 2021-08-02 10:39:10,617] Trial 2 finished with value: 7.452241699416761 and parameters: {'cost_min_beta_w': 0.039977795724596144, 'pixel4_coef': 0.38045876147031077, 'samsungs20ultra_coef': 0.6054769861225261}. Best is trial 1 with value: 7.1314102160395185.
[I 2021-08-02 10:39:10,969] Trial 3 finished with value: 7.014899594696683 and parameters: {'cost_min_be

0.47886964513579444


In [34]:
for phone in train['phone'].unique():
    phonename = phone.split('_')[1]
    if phonename == 'Pixel4':
        w = cost_min_beta_w * pixel4_coef
    elif phonename == 'SamsungS20Ultra':
        w = cost_min_beta_w * samsungs20ultra_coef
    idx = train[train['phone']==phone].index
    train.loc[idx, ['latDeg', 'lngDeg']] = cost_minimization(train.loc[idx], w)

In [35]:
res = train_result(train)
print(res.score)
display(res.err)

6.930960382031107


,percentile50,percentile95,p50_p90_mean
phone,,,
2021-04-22-US-SJC-1_Pixel4,3.573949,13.316521,8.445235
2021-04-22-US-SJC-1_SamsungS20Ultra,3.951059,10.525974,7.238516
2021-04-28-US-SJC-1_Pixel4,2.397350,10.688218,6.542784
2021-04-28-US-SJC-1_SamsungS20Ultra,2.824985,9.169499,5.997242
2021-04-29-US-SJC-2_Pixel4,2.160549,10.678918,6.419734
2021-04-29-US-SJC-2_SamsungS20Ultra,2.786549,11.097953,6.942251


# phones_mean

In [36]:
def calc_mean_pred(df, ws):
    '''
    Generate interpolated lat,lng values for different phone times in the same collection.
    '''
    time_list = df[['collectionName', 'millisSinceGpsEpoch']].drop_duplicates()
    phone_list = df[['collectionName', 'phoneName']].drop_duplicates()
    tmp = time_list.merge(phone_list, on='collectionName', how='outer')
    tmp['phone'] = tmp['collectionName'] + '_' + tmp['phoneName']
    tmp = tmp.merge(df, on=['collectionName', 'phoneName', 'phone', 'millisSinceGpsEpoch'], how='left')
    
    lerp_df = pd.DataFrame()
    for phone in tmp['phone'].unique():
        lerp_tmp = tmp[tmp['phone']==phone].copy()
        lerp_tmp = lerp_tmp.sort_values('millisSinceGpsEpoch')
        lerp_tmp = lerp_tmp.set_index('millisSinceGpsEpoch')
        lerp_tmp[['latDeg', 'lngDeg']] = lerp_tmp[['latDeg', 'lngDeg']].interpolate(method='index', limit_area='inside')
        lerp_tmp = lerp_tmp.reset_index()
        lerp_df = lerp_df.append(lerp_tmp)
    
    lerp_df['w'] = lerp_df['phoneName'].replace(ws)
    lerp_df.loc[lerp_df['latDeg'].isnull(), 'w'] = np.nan
    lerp_df['w_sum'] = lerp_df.groupby(['collectionName', 'millisSinceGpsEpoch'])['w'].transform('sum')
    lerp_df['w'] = lerp_df['w'] / lerp_df['w_sum']
    lerp_df['latDeg'] = lerp_df['latDeg'] * lerp_df['w']
    lerp_df['lngDeg'] = lerp_df['lngDeg'] * lerp_df['w']
    mean_pred = lerp_df.groupby(['collectionName', 'millisSinceGpsEpoch'])['latDeg', 'lngDeg'].sum().reset_index()
    
    output_df = df[['collectionName', 'phoneName', 'millisSinceGpsEpoch']].copy()
    output_df = output_df.merge(mean_pred[['collectionName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['collectionName', 'millisSinceGpsEpoch'], how='left')
    return output_df

In [37]:
def phones_mean_objective(trial):
    Pixel4 = trial.suggest_int('Pixel4', 1, 10)
    SamsungS20Ultra = trial.suggest_int('SamsungS20Ultra', 1, 10)
    
    ws = {'Pixel4':Pixel4, 'SamsungS20Ultra':SamsungS20Ultra}
    
    out = calc_mean_pred(train, ws)
    score = train_result(out).score
    
    return score

In [38]:
study = optuna.create_study()
study.optimize(phones_mean_objective, n_trials=100)
phones_mean_weights = study.best_params
print(phones_mean_weights)

[I 2021-08-02 10:39:28,203] A new study created in memory with name: no-name-b5ee0140-48a6-4760-856a-598bceed26e0
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
[I 2021-08-02 10:39:28,661] Trial 0 finished with value: 6.429734735325624 and parameters: {'Pixel4': 1, 'SamsungS20Ultra': 9}. Best is trial 0 with value: 6.429734735325624.
[I 2021-08-02 10:39:29,110] Trial 1 finished with value: 6.292628939103628 and parameters: {'Pixel4': 1, 'SamsungS20Ultra': 6}. Best is trial 1 with value: 6.292628939103628.
[I 2021-08-02 10:39:29,566] Trial 2 finished with value: 6.221839654319343 and parameters: {'Pixel4': 1, 'SamsungS20Ultra': 5}. Best is trial 2 with value: 6.221839654319343.
[I 2021-08-02 10:39:30,010] Trial 3 finished with value: 5.889849169328588 and parameters: {'Pixel4': 10, 'SamsungS20Ultra': 9}. Best is trial 3 with value: 5.889849169328

{'Pixel4': 6, 'SamsungS20Ultra': 10}


In [39]:
train = calc_mean_pred(train, phones_mean_weights)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [40]:
res = train_result(train)
print(res.score)
display(res.err)

5.879461520018203


,percentile50,percentile95,p50_p90_mean
phone,,,
2021-04-22-US-SJC-1_Pixel4,3.842151,10.279759,7.060955
2021-04-22-US-SJC-1_SamsungS20Ultra,3.751606,10.226122,6.988864
2021-04-28-US-SJC-1_Pixel4,2.556783,7.619251,5.088017
2021-04-28-US-SJC-1_SamsungS20Ultra,2.580141,7.703377,5.141759
2021-04-29-US-SJC-2_Pixel4,2.165617,8.931010,5.548314
2021-04-29-US-SJC-2_SamsungS20Ultra,2.034777,8.862943,5.448860


In [45]:
train['phone'] = train['collectionName'] + '_' + train['phoneName']

# position_shift

In [46]:
def WGS84_to_ECEF(lat, lon, alt):
    # convert to radians
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a    = 6378137.0
    # f is the flattening factor
    finv = 298.257223563
    f = 1 / finv   
    # e is the eccentricity
    e2 = 1 - (1 - f) * (1 - f)    
    # N is the radius of curvature in the prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z

transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)

def ECEF_to_WGS84(x,y,z):
    lon, lat, alt = transformer.transform(x,y,z,radians=False)
    return lon, lat, alt
msge = 'millisSinceGpsEpoch'

In [47]:
def position_shift(df,a):

    d = df.copy()
    d['heightAboveWgs84EllipsoidM'] = 63.5
    d['x'], d['y'], d['z'] = zip(*d.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, x.heightAboveWgs84EllipsoidM), axis=1))

    #a = -0.2
    d.sort_values(['phone', msge], inplace=True)
    for fi in ['x','y','z']:
        d[[fi+'p']] = d[fi].shift().where(d['phone'].eq(d['phone'].shift()))
        d[[fi+'diff']] = d[fi]-d[fi+'p']
    #d[['yp']] = d['y'].shift().where(d['phone'].eq(d['phone'].shift()))
    d[['dist']] = np.sqrt(d['xdiff']**2 + d['ydiff']**2+ d['zdiff']**2)
    for fi in ['x','y','z']:
        d[[fi+'new']] = d[fi+'p'] + d[fi+'diff']*(1-a/d['dist'])
    lng, lat, alt = ECEF_to_WGS84(d['xnew'].values,d['ynew'].values,d['znew'].values)
    
    lng[np.isnan(lng)] = d.loc[np.isnan(lng),'lngDeg']
    lat[np.isnan(lat)] = d.loc[np.isnan(lat),'latDeg']
    d['latDeg'] = lat
    d['lngDeg'] = lng
    
    return d 

def objective(trial):
    a = trial.suggest_uniform('a', -1, 1)
    score = get_train_score(position_shift(train, a),gt)
    return score

In [48]:
study = optuna.create_study()
study.optimize(objective, n_trials=30)
opt_a = study.best_params['a']
print(opt_a)

[I 2021-08-02 10:44:12,683] A new study created in memory with name: no-name-5dd476ef-043a-48e8-bd15-0531507cadc6
[I 2021-08-02 10:44:13,121] Trial 0 finished with value: 5.7089550051733156 and parameters: {'a': 0.8346526627532704}. Best is trial 0 with value: 5.7089550051733156.
[I 2021-08-02 10:44:13,559] Trial 1 finished with value: 5.985254519864249 and parameters: {'a': -0.25629505615487824}. Best is trial 0 with value: 5.7089550051733156.
[I 2021-08-02 10:44:13,996] Trial 2 finished with value: 5.971431145058925 and parameters: {'a': -0.22725401347631857}. Best is trial 0 with value: 5.7089550051733156.
[I 2021-08-02 10:44:14,430] Trial 3 finished with value: 5.761481341878766 and parameters: {'a': 0.4783483723323074}. Best is trial 0 with value: 5.7089550051733156.
[I 2021-08-02 10:44:14,985] Trial 4 finished with value: 5.944979544954877 and parameters: {'a': -0.16396675175681397}. Best is trial 0 with value: 5.7089550051733156.
[I 2021-08-02 10:44:15,418] Trial 5 finished with

0.981107764850147


In [49]:
train = position_shift(train, opt_a)
train.to_csv(OUTPUT + '/train_ro_rog_kf_sp0_pm_ps.csv', index=False)

In [50]:
res = train_result(train)
print(res.score)
display(res.err)

5.689982070190756


,percentile50,percentile95,p50_p90_mean
phone,,,
2021-04-22-US-SJC-1_Pixel4,3.580857,9.490830,6.535843
2021-04-22-US-SJC-1_SamsungS20Ultra,3.490542,9.455580,6.473061
2021-04-28-US-SJC-1_Pixel4,2.586463,7.680867,5.133665
2021-04-28-US-SJC-1_SamsungS20Ultra,2.497249,7.768858,5.133053
2021-04-29-US-SJC-2_Pixel4,2.358536,8.528638,5.443587
2021-04-29-US-SJC-2_SamsungS20Ultra,2.352237,8.489129,5.420683


In [51]:
train_sp0_pred = pd.read_csv(f'{sp0_dir}/train_sp0_pred.csv')
train = sp0_process(train, train_sp0_pred)

In [52]:
res = train_result(train)
print(res.score)
display(res.err)

5.544383022731245


,percentile50,percentile95,p50_p90_mean
phone,,,
2021-04-22-US-SJC-1_Pixel4,3.624078,9.706402,6.665240
2021-04-22-US-SJC-1_SamsungS20Ultra,3.447617,9.474468,6.461043
2021-04-28-US-SJC-1_Pixel4,2.561147,6.998235,4.779691
2021-04-28-US-SJC-1_SamsungS20Ultra,2.456120,7.213236,4.834678
2021-04-29-US-SJC-2_Pixel4,2.230846,8.288086,5.259466
2021-04-29-US-SJC-2_SamsungS20Ultra,2.211344,8.321018,5.266181


In [53]:
train.to_csv(OUTPUT + f'/{nb_name}_train.csv', index=False)

# sub作成

In [54]:
test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
test = test[test['collectionName'].isin(target)]

In [55]:
rb = pd.read_csv(f'{rb_dir}/result_test.csv')

rb = rb.rename(columns={'latDeg':'latDeg_rb', 'lngDeg':'lngDeg_rb'})

test = test.merge(rb[['millisSinceGpsEpoch', 'phone', 'latDeg_rb', 'lngDeg_rb']], on=['millisSinceGpsEpoch', 'phone'], how='left')

idx = test[~test['latDeg_rb'].isnull()].index
test.loc[idx, 'latDeg'] = test.loc[idx, 'latDeg_rb']
test.loc[idx, 'lngDeg'] = test.loc[idx, 'lngDeg_rb']

In [56]:
test = add_distance_diff(test)
test.loc[((test['dist_prev'] > ro_th) & (test['dist_next'] > ro_th)), ['latDeg', 'lngDeg']] = np.nan

In [57]:
rog_df = test[test['collectionName'].isin(rog_target)].copy()
rog_gt = gt[gt['collectionName'].isin(rog_target)].copy()
rog_df = remove_based_on_gt(rog_df, rog_gt, rog_th)
test.loc[rog_df.index, ['latDeg', 'lngDeg']] = rog_df[['latDeg', 'lngDeg']]

In [58]:
test_sp0_pred = pd.read_csv(f'{sp0_dir}/test_sp0_pred.csv')
test = sp0_process(test, test_sp0_pred)

In [59]:
dp_test = pd.read_csv('../output/prep/rel_pred_v001/test_result.csv')
test = test.merge(dp_test[['millisSinceGpsEpoch', 'phone', 'lat_diff', 'lng_diff']], on=['millisSinceGpsEpoch', 'phone'], how='left')
test.loc[test['isSpeed0']==1, ['lat_diff', 'lng_diff']] = 0

In [60]:
test = interpolate_by_rel(test)

In [61]:
test = apply_kf_smoothing(test)

In [62]:
for phone in train['phone'].unique():
    phonename = phone.split('_')[1]
    if phonename == 'Pixel4':
        w = cost_min_beta_w * pixel4_coef
    elif phonename == 'SamsungS20Ultra':
        w = cost_min_beta_w * samsungs20ultra_coef
    idx = train[train['phone']==phone].index
    train.loc[idx, ['latDeg', 'lngDeg']] = cost_minimization(train.loc[idx], w)

KeyError: "None of [Index(['lat_diff', 'lng_diff'], dtype='object')] are in the [columns]"

In [ ]:
test = calc_mean_pred(test, phones_mean_weights)

In [ ]:
test['phone'] = test['collectionName'] + '_' + test['phoneName']
test = position_shift(test, opt_a)

In [ ]:
test_sp0_pred = pd.read_csv(f'{sp0_dir}/test_sp0_pred.csv')
test = sp0_process(test, test_sp0_pred)

In [ ]:
test = test[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']].copy()
test.to_csv(OUTPUT + f'/{nb_name}_test.csv', index=False)

In [ ]:
test